# Predicting VFD in Features_topn_smogned

### Notebook automatically generated from your model

Model XGBoost (All), trained on 2022-01-23 20:45:29.

#### Generated on 2022-01-31 06:32:44.057160

prediction
This notebook will reproduce the steps for a REGRESSION on  Features_topn_smogned.
The main objective is to predict the variable VFD

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [0]:
import sys
import dataiku
import numpy as np
import pandas as pd
import sklearn as sk
import dataiku.core.pandasutils as pdu
from dataiku.doctor.preprocessing import PCA
from collections import defaultdict, Counter

And tune pandas display options:

In [0]:
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#### Importing base data

The first step is to get our machine learning dataset:

In [0]:
# We apply the preparation that you defined. You should not modify this.
preparation_steps = []
preparation_output_schema = {'columns': [{'name': 'StudyID_Int', 'type': 'bigint'}, {'name': 'Sex', 'type': 'string'}, {'name': 'Race', 'type': 'string'}, {'name': 'Infection Status', 'type': 'string'}, {'name': 'ECMO', 'type': 'boolean'}, {'name': 'PELOD', 'type': 'double'}, {'name': 'PRISM', 'type': 'double'}, {'name': 'Age_years', 'type': 'double'}, {'name': 'TNFSF8', 'type': 'double'}, {'name': 'TLR5', 'type': 'double'}, {'name': 'SYK', 'type': 'double'}, {'name': 'PRDM1', 'type': 'double'}, {'name': 'LTB4R', 'type': 'double'}, {'name': 'LILRA3', 'type': 'double'}, {'name': 'IRF7', 'type': 'double'}, {'name': 'IL6', 'type': 'double'}, {'name': 'IL1RN', 'type': 'double'}, {'name': 'IL1R2', 'type': 'double'}, {'name': 'IL18RAP', 'type': 'double'}, {'name': 'IL16', 'type': 'double'}, {'name': 'IKZF2', 'type': 'double'}, {'name': 'HAMP', 'type': 'double'}, {'name': 'FKBP5', 'type': 'double'}, {'name': 'FCGR3A/B', 'type': 'double'}, {'name': 'EGR1', 'type': 'double'}, {'name': 'DPP4', 'type': 'double'}, {'name': 'CXCR1', 'type': 'double'}, {'name': 'CSF2RB', 'type': 'double'}, {'name': 'CLEC4E', 'type': 'double'}, {'name': 'CFD', 'type': 'double'}, {'name': 'CD9', 'type': 'double'}, {'name': 'VFD', 'type': 'bigint'}], 'userModified': False}

ml_dataset_handle = dataiku.Dataset('Features_topn_smogned')
ml_dataset_handle.set_preparation_steps(preparation_steps, preparation_output_schema)
%time ml_dataset = ml_dataset_handle.get_dataframe(limit = 100000)

print ('Base data has %i rows and %i columns' % (ml_dataset.shape[0], ml_dataset.shape[1]))
# Five first records",
ml_dataset.head(5)

#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

In [0]:
ml_dataset = ml_dataset[['CFD', 'IL1RN', 'Sex', 'CSF2RB', 'PELOD', 'PRDM1', 'IKZF2', 'LILRA3', 'LTB4R', 'DPP4', 'IL18RAP', 'CXCR1', 'Infection Status', 'VFD', 'Race', 'FCGR3A/B', 'ECMO', 'EGR1', 'SYK', 'IL1R2', 'IL16', 'PRISM', 'IL6', 'Age_years', 'IRF7', 'CD9', 'TNFSF8', 'TLR5', 'CLEC4E', 'HAMP', 'FKBP5']]

Let's first coerce categorical columns into unicode, numerical features into floats.

In [0]:
# astype('unicode') does not work as expected

def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = ['Sex', 'Infection Status', 'Race', 'ECMO']
numerical_features = ['CFD', 'IL1RN', 'CSF2RB', 'PELOD', 'PRDM1', 'IKZF2', 'LILRA3', 'LTB4R', 'DPP4', 'IL18RAP', 'CXCR1', 'FCGR3A/B', 'EGR1', 'SYK', 'IL1R2', 'IL16', 'PRISM', 'IL6', 'Age_years', 'IRF7', 'CD9', 'TNFSF8', 'TLR5', 'CLEC4E', 'HAMP', 'FKBP5']
text_features = []
from dataiku.doctor.utils import datetime_to_epoch
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    if ml_dataset[feature].dtype == np.dtype('M8[ns]') or (hasattr(ml_dataset[feature].dtype, 'base') and ml_dataset[feature].dtype.base == np.dtype('M8[ns]')):
        ml_dataset[feature] = datetime_to_epoch(ml_dataset[feature])
    else:
        ml_dataset[feature] = ml_dataset[feature].astype('double')

We renamed the target variable to a column named target

In [0]:
ml_dataset['__target__'] = ml_dataset['VFD']
del ml_dataset['VFD']


# Remove rows for which the target is unknown.
ml_dataset = ml_dataset[~ml_dataset['__target__'].isnull()]

#### Cross-validation strategy

The dataset needs to be split into 2 new sets, one that will be used for training the model (train set)
and another that will be used to test its generalization capability (test set)

This is a simple cross-validation strategy.

In [0]:
train, test = pdu.split_train_valid(ml_dataset, prop=0.8)
print ('Train data has %i rows and %i columns' % (train.shape[0], train.shape[1]))
print ('Test data has %i rows and %i columns' % (test.shape[0], test.shape[1]))

#### Features preprocessing

The first thing to do at the features level is to handle the missing values.
Let's reuse the settings defined in the model

In [0]:
drop_rows_when_missing = []
impute_when_missing = [{'feature': 'CFD', 'impute_with': 'MEAN'}, {'feature': 'IL1RN', 'impute_with': 'MEAN'}, {'feature': 'CSF2RB', 'impute_with': 'MEAN'}, {'feature': 'PELOD', 'impute_with': 'MEAN'}, {'feature': 'PRDM1', 'impute_with': 'MEAN'}, {'feature': 'IKZF2', 'impute_with': 'MEAN'}, {'feature': 'LILRA3', 'impute_with': 'MEAN'}, {'feature': 'LTB4R', 'impute_with': 'MEAN'}, {'feature': 'DPP4', 'impute_with': 'MEAN'}, {'feature': 'IL18RAP', 'impute_with': 'MEAN'}, {'feature': 'CXCR1', 'impute_with': 'MEAN'}, {'feature': 'FCGR3A/B', 'impute_with': 'MEAN'}, {'feature': 'EGR1', 'impute_with': 'MEAN'}, {'feature': 'SYK', 'impute_with': 'MEAN'}, {'feature': 'IL1R2', 'impute_with': 'MEAN'}, {'feature': 'IL16', 'impute_with': 'MEAN'}, {'feature': 'PRISM', 'impute_with': 'MEAN'}, {'feature': 'IL6', 'impute_with': 'MEAN'}, {'feature': 'Age_years', 'impute_with': 'MEAN'}, {'feature': 'IRF7', 'impute_with': 'MEAN'}, {'feature': 'CD9', 'impute_with': 'MEAN'}, {'feature': 'TNFSF8', 'impute_with': 'MEAN'}, {'feature': 'TLR5', 'impute_with': 'MEAN'}, {'feature': 'CLEC4E', 'impute_with': 'MEAN'}, {'feature': 'HAMP', 'impute_with': 'MEAN'}, {'feature': 'FKBP5', 'impute_with': 'MEAN'}]

# Features for which we drop rows with missing values"
for feature in drop_rows_when_missing:
    train = train[train[feature].notnull()]
    test = test[test[feature].notnull()]
    print ('Dropped missing records in %s' % feature)

# Features for which we impute missing values"
for feature in impute_when_missing:
    if feature['impute_with'] == 'MEAN':
        v = train[feature['feature']].mean()
    elif feature['impute_with'] == 'MEDIAN':
        v = train[feature['feature']].median()
    elif feature['impute_with'] == 'CREATE_CATEGORY':
        v = 'NULL_CATEGORY'
    elif feature['impute_with'] == 'MODE':
        v = train[feature['feature']].value_counts().index[0]
    elif feature['impute_with'] == 'CONSTANT':
        v = feature['value']
    train[feature['feature']] = train[feature['feature']].fillna(v)
    test[feature['feature']] = test[feature['feature']].fillna(v)
    print ('Imputed missing values in feature %s with value %s' % (feature['feature'], coerce_to_unicode(v)))

We can now handle the categorical features (still using the settings defined in Models):

Let's dummy-encode the following features.
A binary column is created for each of the 100 most frequent values.

In [0]:
LIMIT_DUMMIES = 100

categorical_to_dummy_encode = ['Sex', 'Infection Status', 'Race', 'ECMO']

# Only keep the top 100 values
def select_dummy_values(train, features):
    dummy_values = {}
    for feature in categorical_to_dummy_encode:
        values = [
            value
            for (value, _) in Counter(train[feature]).most_common(LIMIT_DUMMIES)
        ]
        dummy_values[feature] = values
    return dummy_values

DUMMY_VALUES = select_dummy_values(train, categorical_to_dummy_encode)

def dummy_encode_dataframe(df):
    for (feature, dummy_values) in DUMMY_VALUES.items():
        for dummy_value in dummy_values:
            dummy_name = u'%s_value_%s' % (feature, coerce_to_unicode(dummy_value))
            df[dummy_name] = (df[feature] == dummy_value).astype(float)
        del df[feature]
        print ('Dummy-encoded feature %s' % feature)

dummy_encode_dataframe(train)

dummy_encode_dataframe(test)

Let's rescale numerical features

In [0]:
rescale_features = {'CFD': 'AVGSTD', 'IL1RN': 'AVGSTD', 'CSF2RB': 'AVGSTD', 'PELOD': 'AVGSTD', 'PRDM1': 'AVGSTD', 'IKZF2': 'AVGSTD', 'LILRA3': 'AVGSTD', 'LTB4R': 'AVGSTD', 'DPP4': 'AVGSTD', 'IL18RAP': 'AVGSTD', 'CXCR1': 'AVGSTD', 'FCGR3A/B': 'AVGSTD', 'EGR1': 'AVGSTD', 'SYK': 'AVGSTD', 'IL1R2': 'AVGSTD', 'IL16': 'AVGSTD', 'PRISM': 'AVGSTD', 'IL6': 'AVGSTD', 'Age_years': 'AVGSTD', 'IRF7': 'AVGSTD', 'CD9': 'AVGSTD', 'TNFSF8': 'AVGSTD', 'TLR5': 'AVGSTD', 'CLEC4E': 'AVGSTD', 'HAMP': 'AVGSTD', 'FKBP5': 'AVGSTD'}
for (feature_name, rescale_method) in rescale_features.items():
    if rescale_method == 'MINMAX':
        _min = train[feature_name].min()
        _max = train[feature_name].max()
        scale = _max - _min
        shift = _min
    else:
        shift = train[feature_name].mean()
        scale = train[feature_name].std()
    if scale == 0.:
        del train[feature_name]
        del test[feature_name]
        print ('Feature %s was dropped because it has no variance' % feature_name)
    else:
        print ('Rescaled %s' % feature_name)
        train[feature_name] = (train[feature_name] - shift).astype(np.float64) / scale
        test[feature_name] = (test[feature_name] - shift).astype(np.float64) / scale

#### Modeling

Before actually creating our model, we need to split the datasets into their features and labels parts:

In [0]:
X_train = train.drop('__target__', axis=1)
X_test = test.drop('__target__', axis=1)

y_train = np.array(train['__target__'])
y_test = np.array(test['__target__'])

Now we can finally create our model!

In [0]:
import xgboost as xgb
clf = xgb.XGBRegressor(
                    max_depth=3,
                    learning_rate=0.2,
                    gamma=0.0,
                    min_child_weight=1.0,
                    max_delta_step=0.0,
                    subsample=1.0,
                    colsample_bytree=1.0,
                    colsample_bylevel=1.0,
                    reg_alpha=0.0,
                    reg_lambda=1.0,
                    n_estimators=19,
                    silent=0,
                    nthread=4,
                    scale_pos_weight=1.0,
                    base_score=0.5,
                    seed=1337,
                    missing=None,
                  )

... And train the model

In [0]:
%time clf.fit(X_train, y_train)

Build up our result dataset

In [0]:
%time _predictions = clf.predict(X_test)
predictions = pd.Series(data=_predictions, index=X_test.index, name='predicted_value')

# Build scored dataset
results_test = X_test.join(predictions, how='left')
results_test = results_test.join(test['__target__'], how='left')
results_test = results_test.rename(columns= {'__target__': 'VFD'})

#### Results

You can measure the model's accuracy:

In [0]:
c =  results_test[['predicted_value', 'VFD']].corr()
print ('Pearson correlation: %s' % c['predicted_value'][1])

That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
